In [1]:
# Google Colab preparation

%%capture

!pip install scikit-optimize
!pip install imblearn

!git clone https://github.com/ReadingHui/tabular-playground-s4e10-loan-approval

import sys
sys.path.insert(0,'/content/tabular-playground-s4e10-loan-approval')

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from imblearn.over_sampling import RandomOverSampler

import matplotlib.pyplot as plt
import preprocess

import torch
from torch import nn
from torch import optim
import torch.utils.data as data_utils
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Global Variables

In [3]:
# In Google Colab
train_path = '/content/tabular-playground-s4e10-loan-approval/train.csv'
test_path = '/content/tabular-playground-s4e10-loan-approval/test.csv'

# # In Local machine
# train_path = 'train.csv'
# test_path = 'test.csv'

random_state = 1048576
target = 'loan_status'
cat_features = [
    'person_home_ownership',
    'loan_intent',
    'loan_grade',
    'cb_person_default_on_file'
]

num_features = [
    'person_age',
    'person_income',
    'person_emp_length',
    'loan_amnt',
    'loan_int_rate',
    'loan_percent_income',
    'cb_person_cred_hist_length'
]

# Data Loading and Preprocessing

In [4]:
# Data Loading
X_train, X_test, y_train, y_test = preprocess.DataImport.get_train_test(
    train_path, index_col=0, target=target,
    random_state=random_state, verbose=1
    )

# Remove outliers
X_train, y_train = preprocess.Outliers.remove_outliers(
    X_train, y_train, verbose=1
    )

# One-Hot Encode
encoder = preprocess.CategoryEncoder(cat_features=cat_features,
                                     method='one_hot')
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

# Standard Scaling
scaler = StandardScaler()
features = X_train.columns
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=features)
X_test = pd.DataFrame(scaler.transform(X_test), columns=features)

# Oversampling
print(f'Target distribution before oversampling: ',
      f'(0: {round(y_train.sum() / y_train.size * 100, 2)}%, ',
      f'1: {round((1 - y_train.sum() / y_train.size) * 100, 2)}%)')
oversampler = RandomOverSampler(random_state=random_state)
X_train, y_train = oversampler.fit_resample(X_train, y_train)
print(f'X_train shape after oversampling: {X_train.shape}; ',
      f'y_train shape after oversampling: {X_train.shape}')
print(f'Target distribution after oversampling: ',
      f'(0: {round(y_train.sum() / y_train.size * 100, 2)}%, ',
      f'1: {round((1 - y_train.sum() / y_train.size) * 100, 2)}%)')

# Train - Validation Split
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.1, random_state=random_state)

print(f'Shape of X_train is: {X_train.shape}; ',
      f'shape of y_train is: {y_train.shape}')
print(f'Shape of X_val is: {X_val.shape}; ',
      f'shape of y_test is: {y_val.shape}')

# Casting everything to pytorch tensor
X_train = torch.tensor(X_train.values, device=device, dtype=torch.float32)
y_train = torch.tensor(y_train.values, device=device, dtype=torch.float32)
X_val = torch.tensor(X_val.values, device=device, dtype=torch.float32)
y_val = torch.tensor(y_val.values, device=device, dtype=torch.float32)
X_test = torch.tensor(X_test.values, device=device, dtype=torch.float32)
y_test = torch.tensor(y_test.values, device=device, dtype=torch.float32)

Shape of X_train is: (52780, 11); shape of y_train is: (52780,)
Shape of X_test is: (5865, 11); shape of y_test is: (5865,)
New shape of X_train is: (52639, 11); New shape of y_train is: (52639,)
Target distribution before oversampling: (0: 14.22%, 1: 85.78%)
X_train shape after oversampling: (90312, 26); y_train shape after oversampling: (90312, 26)
Target distribution after oversampling: (0: 50.0%, 1: 50.0%)
Shape of X_train is: (81280, 26); shape of y_train is: (81280,)
Shape of X_val is: (9032, 26); shape of y_test is: (9032,)


# Model building

In [5]:
# Define the neural network model
class BinaryClassificationNN(nn.Module):
    def __init__(self):
        super(BinaryClassificationNN, self).__init__()

        # Define the layers
        self.fc1 = nn.Linear(in_features=26, out_features=32)
        self.fc2 = nn.Linear(in_features=32, out_features=16)
        self.fc3 = nn.Linear(in_features=16, out_features=8)
        self.fc4 = nn.Linear(in_features=8, out_features=1)

        # Activation function
        self.relu = nn.ReLU()

        # Sigmoid for probability output
        self.sigmoid = nn.Sigmoid()

        # Initialize weights
        self.init_weights()  # Custom weight initialization

    def init_weights(self):
        # Initialize weights with Xavier (Glorot) initialization
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.xavier_uniform_(self.fc4.weight)

        # Optionally, initialize biases to zero
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)
        nn.init.zeros_(self.fc4.bias)


    def forward(self, X: pd.DataFrame | np.ndarray) -> pd.DataFrame | np.ndarray:
        X = self.relu(self.fc1(X))
        X = self.relu(self.fc2(X))
        X = self.relu(self.fc3(X))
        X = self.sigmoid(self.fc4(X))  # Sigmoid for binary classification
        return X



# Model Training

In [6]:
def save_checkpoint(model: nn.Module, optimizer: optim.Optimizer,
                    epoch: int, loss: float, score: float,
                    filename: str = 'checkpoint.pth', verbose: bool = False):
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'score': score
    }, filename)
    if verbose:
        print(f'Checkpoint saved at epoch {epoch + 1}')

def load_checkpoint(filename: str = 'checkpoint.pth'):
    model = BinaryClassificationNN()
    optimizer = optim.Adam(model.parameters())
    checkpoint = torch.load(filename, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    return model, optimizer, epoch, loss

In [7]:
class EarlyStopping:
    def __init__(self, patience: int = 5, method: str = 'loss'):
        self.patience = patience
        self.method = method
        self.counter = 0
        self.best_value = np.inf
        self.early_stop = False

    def __call__(self, val_value):
        if self.method == 'loss':
            val_value = val_value
        elif self.method == 'score':
            val_value = -val_value
        if val_value < self.best_value:
            self.best_value = val_value
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [8]:
def train_model(model: nn.Module,
                X: torch.Tensor, y: torch.Tensor,
                eval_set: tuple[torch.Tensor] = None,
                num_epochs: int = 10, batch_size: int = 32,
                shuffle: bool = True, learning_rate: float = 0.01,
                weight_decay: float = 1e-3,
                criterion: torch.nn.modules.loss = nn.BCELoss(),
                early_stopping_method: str = 'loss',
                early_stopping_patience: int = 5,
                save_path: str = None,
                verbose: int = 1):
    # Initialize optimizer, early_stopping and history parameters
    optimizer = optim.Adam(model.parameters(),
                           lr=learning_rate,
                           weight_decay=weight_decay)
    early_stopping = EarlyStopping(patience=early_stopping_patience,
                                   method=early_stopping_method)
    best_loss = np.inf

    losses = {"train": [], "val": []}
    scores = {"train": [], "val": []}

    # Initialize data_laoder
    X = X.clone()
    y = y.clone()
    y_train_labels = y.cpu() if next(model.parameters()).is_cuda else y
    train_tensor = data_utils.TensorDataset(X, y)
    train_loader = data_utils.DataLoader(dataset=train_tensor,
                                         batch_size=batch_size,
                                         shuffle=shuffle)
    if eval_set:
        X_val = eval_set[0].clone()
        y_val = eval_set[1].clone()
        y_val_labels = y_val.cpu() if next(model.parameters()).is_cuda \
        else y_val
        val_tensor = data_utils.TensorDataset(X_val, y_val)
        val_loader = data_utils.DataLoader(dataset=val_tensor,
                                           batch_size=batch_size,
                                           shuffle=shuffle)

    for epoch in range(num_epochs):  # Training loop
            # Training phrase
            total_train_loss = 0

            model.train()  # Set the model to training mode
            for batch_idx, (inputs, labels) in enumerate(train_loader):
                # Forward pass
                outputs = model(inputs)
                outputs = torch.squeeze(outputs)

                # Compute the loss
                loss = criterion(outputs, labels)

                # Backpropagation and optimization
                optimizer.zero_grad()  # Clear previous gradients
                loss.backward() # Backpropagate
                optimizer.step() # Update weights

                # Rolling sum of training loss
                total_train_loss += loss.item()

                # If large verbose print batch loss
                if verbose > 1:
                    print_freq = num_epochs // 10
                    if batch_idx % print_freq == 0:
                        print(f"Epoch [{epoch+1}/{num_epochs}], \
                        Batch [{batch_idx}], \
                        Loss: {loss.item():.4f}") # Printing loss

            # Store the loss value for this epoch
            average_train_loss = round(total_train_loss / len(train_loader), 4)
            losses['train'].append(average_train_loss)

            # Evaluate training score
            with torch.no_grad():
                y_train_pred = model(X).detach()
                if next(model.parameters()).is_cuda:
                    y_train_pred = y_train_pred.cpu()
                y_train_pred = y_train_pred.numpy()
            train_score = round(roc_auc_score(y_train_labels, y_train_pred), 4)
            scores['train'].append(train_score)
            if verbose > 0 and not eval_set:
                print(f"Epoch [{epoch+1}/{num_epochs}], ",
                    f"Average Train Loss: {average_train_loss}, ",
                    f"Training Score: {train_score}.") # Printing loss

            if eval_set:
                total_val_loss = 0 # Initiate total_val loss

                model.eval()  # Set the model to evaluation mode
                # Evaluate the model on the validation set
                for inputs, labels in val_loader: # For each batch
                # Forward pass
                    outputs = model(inputs)
                    outputs = torch.squeeze(outputs)

                    # Compute the loss
                    loss = criterion(outputs, labels)
                    total_val_loss += loss.item() # Rolling sum of training loss

                # Store the loss value for this epoch
                average_val_loss = round(total_val_loss / len(val_loader), 4)
                losses['val'].append(average_val_loss)

                # Evaluate validation score
                with torch.no_grad():
                    y_val_pred = model(X_val).detach()
                    if next(model.parameters()).is_cuda:
                        y_val_pred = y_val_pred.cpu()
                    y_val_pred = y_val_pred.numpy()
                val_score = round(roc_auc_score(y_val_labels, y_val_pred), 4)
                scores['val'].append(val_score)

                if verbose > 0:
                    print(f"Epoch [{epoch+1}/{num_epochs}], ",
                    f"Average Train Loss: {average_train_loss}, ",
                    f"Training Score: {train_score}, ",
                    f"Average Val Loss: {average_val_loss}, ",
                    f"Validation Score: {val_score}.") # Printing loss

            # Update best model
            if eval_set:
                compare_set = 'val'
            else:
                compare_set = 'train'
            if early_stopping_method == 'loss':
                compare_val = losses[compare_set][-1]
            elif early_stopping_method == 'score':
                compare_val = - scores[compare_set][-1]
            if compare_val < best_loss:
                best_loss = average_val_loss
                save_checkpoint(model, optimizer, epoch,
                                losses[compare_set][-1],
                                scores[compare_set][-1],
                                save_path, verbose)


            # Early Stopping
            if early_stopping_method == 'loss':
                early_stopping(average_val_loss)
            elif early_stopping_method == 'score':
                early_stopping(val_score)
            if early_stopping.early_stop:
                print(f"Early stopping triggered, ",
                      f"validation {early_stopping_method} not improved ",
                      f"after {early_stopping_patience} epochs")
                break
    # Pack history
    history = {}
    history['loss'] = losses
    history['scores'] = scores

    return history

In [ ]:
num_epochs = 1000

torch.manual_seed(random_state)
model = BinaryClassificationNN()
model = model.to(device)
history = train_model(model,
                      X_train, y_train,
                       (X_val, y_val),
                      num_epochs,
                      learning_rate=0.001,
                      save_path='PyTorch_best_model.pt')

Epoch [1/1000],  Average Train Loss: 0.361,  Training Score: 0.9294,  Average Val Loss: 0.3427,  Validation Score: 0.9239.
Checkpoint saved at epoch 1
Epoch [2/1000],  Average Train Loss: 0.3248,  Training Score: 0.9331,  Average Val Loss: 0.3303,  Validation Score: 0.9279.
Checkpoint saved at epoch 2
Epoch [3/1000],  Average Train Loss: 0.3185,  Training Score: 0.9345,  Average Val Loss: 0.3263,  Validation Score: 0.929.
Checkpoint saved at epoch 3
Epoch [4/1000],  Average Train Loss: 0.3152,  Training Score: 0.9352,  Average Val Loss: 0.3244,  Validation Score: 0.9298.
Checkpoint saved at epoch 4
Epoch [5/1000],  Average Train Loss: 0.3126,  Training Score: 0.9358,  Average Val Loss: 0.3229,  Validation Score: 0.9296.
Checkpoint saved at epoch 5
Epoch [6/1000],  Average Train Loss: 0.311,  Training Score: 0.9372,  Average Val Loss: 0.3218,  Validation Score: 0.9315.
Checkpoint saved at epoch 6
Epoch [7/1000],  Average Train Loss: 0.3092,  Training Score: 0.9374,  Average Val Loss: 0.

# Loss analysis

In [ ]:
def plot_loss(history, loss: bool = True, score: bool = False):
    num_epochs = len(history['loss']['train'])
    if not loss and not score:
        raise ValueError('Cannot have both loss and score set as False')

    if loss:
        plt.plot(range(1, num_epochs+1), history['loss']['train'], marker='o')
        plt.plot(range(1, num_epochs+1), history['loss']['val'], marker='o')
    if score:
        plt.plot(range(1, num_epochs+1), history['scores']['train'], marker='x')
        plt.plot(range(1, num_epochs+1), history['scores']['val'], marker='x')

    if loss and score:
        plt.title('Training Loss and Scores Over Epochs')
    elif score:
        plt.title('Scores Over Epochs')
    elif loss:
        plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()

In [ ]:
plot_loss(history)

In [ ]:
best_model, optimizer, epoch, loss = load_checkpoint('PyTorch_best_model.pt')

In [ ]:
best_model = best_model.to(device)
best_model.eval()
with torch.no_grad():
    y_pred = best_model(X_test).detach()
    if next(best_model.parameters()).is_cuda:
        y_pred = y_pred.cpu()
    y_pred = y_pred.numpy()
if next(best_model.parameters()).is_cuda:
    y_test = y_test.cpu()
print(f'Test Score: {round(roc_auc_score(y_test, y_pred), 4)}')

In [ ]:
# Auto terminate session

from google.colab import runtime
runtime.unassign()